# Open Analytics Framework - User Environment Sharing and Re-creation.

### Disclaimer
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.

### Introduction
The `snapshot()` function allows users to capture the current state of their environment in a snapshot file. This file can later be passed to `create_env()` to create an environment with same libraries and files.
* Users can clone the environments by sharing the file with others.
* Before passing the file to create_env(), users can modify the file so the corresponding changes are effective in new environment.
* Files can not be cloned unless user specifies their path in the snapshot file.

Steps to capture the snapshot and re-create the exact same environment:
1. Import the required teradataml modules.
2. Connect to a Vantage system.
3. Set parameters for OpenAF services.
4. Create new environment with installing packages and files in it.
5. Take a snapshot for the environment.
6. Remove created environment.
7. Create environment using snapshot.
8. Cleanup.

## 1. Import the required teradataml modules

In [1]:
from teradataml import create_context, remove_context, set_auth_token, list_base_envs, create_env, remove_env, list_user_envs
import getpass, os, json

## 2. Connect to a Vantage system

In [2]:
# Create a connetion.
host = getpass.getpass("Host: ")
username = getpass.getpass("Username: ")
password = getpass.getpass("Password: ")

con = create_context(host=host, username=username, password=password)

Host:  ········
Username:  ········
Password:  ········


## 3. Set parameters for OpenAF services

In [3]:
# set parameters for OpenAF services.
ues_url = getpass.getpass("ues_url: ")
auth_token = getpass.getpass("auth_token: ")


ues_url:  ········
auth_token:  ········


In [4]:
set_auth_token(ues_url=ues_url, auth_token=auth_token)

True

## 4. Create new environment with installing packages and files in it

#### First we will check what are the available base user environments using `list_base_envs()` and existing user environments using `list_user_envs()`.

User environments are created from available base environments.

In [5]:
list_base_envs()

,base_name,language,version
0,python_3.8,Python,3.8.18
1,python_3.9,Python,3.9.18
2,python_3.10,Python,3.10.13
3,r_4.1,R,4.1.3
4,r_4.0,R,4.0.5
5,r_4.2,R,4.2.2


In [9]:
list_user_envs()

No user environment(s) found.


#### Create environment with 'python_3.10' base_env

In [10]:
env = create_env(env_name="demo_env", base_env="python_3.10", desc="Demo env.")

User environment 'demo_env' created.


#### Install 'scikit-learn' and 'nltk' libraries in 'demo_env' environment.

In [11]:
env.install_lib(["scikit-learn", "nltk"], asynchronous=True)

Request to install libraries initiated successfully in the remote user environment demo_env. Check the status using status() with the claim id 'b382b132-f718-4c90-86e6-782b928dae96'.


'b382b132-f718-4c90-86e6-782b928dae96'

#### Check the status of asynchronous package installation task using claim_id returned by `install_lib()`

In [13]:
env.status("b382b132-f718-4c90-86e6-782b928dae96")

,Claim Id,File/Libs/Model,Method Name,Stage,Timestamp,Additional Details
0,b382b132-f718-4c90-86e6-782b928dae96,"scikit-learn, nltk",install_lib,Started,2024-09-24T06:41:07Z,


#### Let's check the status again

In [16]:
env.status("b382b132-f718-4c90-86e6-782b928dae96")

,Claim Id,File/Libs/Model,Method Name,Stage,Timestamp,Additional Details
0,b382b132-f718-4c90-86e6-782b928dae96,"scikit-learn, nltk",install_lib,Started,2024-09-24T06:41:07Z,
1,b382b132-f718-4c90-86e6-782b928dae96,"scikit-learn, nltk",install_lib,Finished,2024-09-24T06:43:07Z,


#### List the installed libraries in 'demo_env' environment.

In [17]:
env.libs

,name,version
0,click,8.1.7
1,joblib,1.4.2
2,nltk,3.9.1
3,numpy,2.1.1
4,pip,23.0.1
5,regex,2024.9.11
6,scikit-learn,1.5.2
7,scipy,1.14.1
8,setuptools,65.5.0
9,threadpoolctl,3.5.0


#### Install 'mapper.py' file in 'demo_env' environment

In [18]:
env.install_file("/tmp/mapper.py")

File 'mapper.py' installed successfully in the remote user environment 'demo_env'.


True

#### List the installed file in 'demo_env' environment

In [19]:
env.files

,File,Size,Timestamp
0,mapper.py,532,2024-09-24T06:43:43Z


## 5. Take a snapshot for the environment

In [20]:
env.snapshot("/tmp")

Snapshot for environment "demo_env" is stored at "/tmp/snapshot_demo_env_python_3.10_1727160231.8639364.json"


#### Display the content of snapshot file

In [21]:
snapshot_file_path = "/tmp/snapshot_demo_env_python_3.10_1727160231.8639364.json"

In [22]:
with open(snapshot_file_path, 'r') as file:
    data = json.load(file)

print(json.dumps(data, indent=4))

{
    "env_specs": [
        {
            "env_name": "demo_env",
            "base_env": "python_3.10",
            "desc": "Demo env.",
            "libs": [
                "click==8.1.7",
                "joblib==1.4.2",
                "nltk==3.9.1",
                "numpy==2.1.1",
                "pip==23.0.1",
                "regex==2024.9.11",
                "scikit-learn==1.5.2",
                "scipy==1.14.1",
                "setuptools==65.5.0",
                "threadpoolctl==3.5.0",
                "tqdm==4.66.5"
            ],
            "files": [
                "<ADD_YOUR_LOCAL_FILE_PATH>/mapper.py"
            ]
        }
    ]
}


#### Update the local file path in the snapshot file

To recreate the exact same environment, the files referenced in the snapshot must be available on the user's local machine. The user should update the snapshot file with the absolute file paths of referenced files

In [23]:
data["env_specs"][0]['files'][0] = "/tmp/mapper.py"

# Write the updated data back to the JSON file
with open(snapshot_file_path, 'w') as file:
    json.dump(data, file, indent=4)

#### Display the updated content of snapshot file

In [24]:
with open(snapshot_file_path, 'r') as file:
    data = json.load(file)

print(json.dumps(data, indent=4))


{
    "env_specs": [
        {
            "env_name": "demo_env",
            "base_env": "python_3.10",
            "desc": "Demo env.",
            "libs": [
                "click==8.1.7",
                "joblib==1.4.2",
                "nltk==3.9.1",
                "numpy==2.1.1",
                "pip==23.0.1",
                "regex==2024.9.11",
                "scikit-learn==1.5.2",
                "scipy==1.14.1",
                "setuptools==65.5.0",
                "threadpoolctl==3.5.0",
                "tqdm==4.66.5"
            ],
            "files": [
                "/tmp/mapper.py"
            ]
        }
    ]
}


## 6. Remove created environment

In [25]:
remove_env("demo_env")

User environment 'demo_env' removed.


True

## Add new libraries and files in the snapshot file before creating enviroment

#### If you want to install new libraries and files in the environment before recreating it, update the snapshot file accordingly

##### Add new files and libraries in snapshot file

In [26]:
new_file_path = "/tmp/mapper.R"

# Add new file to the list.
data['env_specs'][0]['files'].append(new_file_path)

# Add new libs to the list.
data['env_specs'][0]['libs'].extend(["cryptography", "pyjwt"])

# Save the updated JSON content back to the file
with open(snapshot_file_path, 'w') as file:
    json.dump(data, file, indent=4)

# Display the updated content of the snapshot file
print(json.dumps(data, indent=4))


{
    "env_specs": [
        {
            "env_name": "demo_env",
            "base_env": "python_3.10",
            "desc": "Demo env.",
            "libs": [
                "click==8.1.7",
                "joblib==1.4.2",
                "nltk==3.9.1",
                "numpy==2.1.1",
                "pip==23.0.1",
                "regex==2024.9.11",
                "scikit-learn==1.5.2",
                "scipy==1.14.1",
                "setuptools==65.5.0",
                "threadpoolctl==3.5.0",
                "tqdm==4.66.5",
                "cryptography",
                "pyjwt"
            ],
            "files": [
                "/tmp/mapper.py",
                "/tmp/mapper.R"
            ]
        }
    ]
}


## 7. Create environment using snapshot

####  Only one environment is creating because the snapshot file contains information for a single environment.

In [27]:
create_env(template=snapshot_file_path)

Creating environment 'demo_env'...
User environment 'demo_env' created.
An empty environment 'demo_env' is created.
Installing files in environment 'demo_env'...
File 'mapper.py' installed successfully in the remote user environment 'demo_env'.
File 'mapper.R' installed successfully in the remote user environment 'demo_env'.
File installation in environment 'demo_env' - Completed.
Installing libraries in environment 'demo_env'...
Libraries installation in environment 'demo_env' - Completed.
Created environment 'demo_env' with specified requirements.

Environment Name: demo_env
Base Environment: python_3.10
Description: Demo env.

############ Files installed in User Environment ############

        File Size             Timestamp
0  mapper.py  532  2024-09-24T06:46:26Z
1   mapper.R  594  2024-09-24T06:46:26Z

############ Libraries installed in User Environment ############

             name    version
0            cffi     1.17.1
1           click      8.1.7
2    cryptography     43


Environment Name: demo_env
Base Environment: python_3.10
Description: Demo env.

############ Files installed in User Environment ############

        File Size             Timestamp
0  mapper.py  532  2024-09-24T06:46:26Z
1   mapper.R  594  2024-09-24T06:46:26Z

############ Libraries installed in User Environment ############

             name    version
0            cffi     1.17.1
1           click      8.1.7
2    cryptography     43.0.1
3          joblib      1.4.2
4            nltk      3.9.1
5           numpy      2.1.1
6             pip     23.0.1
7       pycparser       2.22
8           PyJWT      2.9.0
9           regex  2024.9.11
10   scikit-learn      1.5.2
11          scipy     1.14.1
12     setuptools     65.5.0
13  threadpoolctl      3.5.0
14           tqdm     4.66.5


### List the user environments

In [28]:
list_user_envs()

,env_name,env_description,base_env_name,language,conda
0,demo_env,Demo env.,python_3.10,Python,False


## 8. Cleanup

In [29]:
# Remove the re-created environment.
remove_env("demo_env")

User environment 'demo_env' removed.


True

In [30]:
# Removes the current context associated with the Teradata Vantage.
remove_context()

True